# Data --> CSP-Structure

## How should the Data-Structure look like?

It's a Pandas Dataframe with all relevant Information.
| id | size | variables | domains | constraints |
|----|------|-----------|---------|-------------|
| lgp-test-5x6-16 | 5*6 | ['name', 'nationalities', 'book genres', 'lunch', 'color', 'animals'] | {'name': [`Peter`, `Alice`, `Bob`, `Eric`, `Arnold`], 'nationalities': [`norwegian`, `german`, `dane`, `brit`, `swede`], 'book genres': [`fantasy`, `biography`, `romance`, `mystery`, `science fiction`], 'lunch': [`stir fry`, `grilled cheese`, `pizza`, `spaghetti`, `stew`], 'color': [`red`, `green`, `blue`, `yellow`, `white`], 'animals': [`bird`, `dog`, `cat`, `horse`, `fish`]} | [IsContraint('fantasy', 'norwegian'), NextToConstraint('cat', 'biography'), IsConstraint('German', 'Bob'), IsConstraint('yellow', 'Bob'), IsConstraint('Green', 'Peter'), LeftOrRightConstraint('Dane', 'Pizza', distance=1), LeftConstraint('blue', 'Dane'), LeftConstraint('grilled Cheese', 'Norwegian'), ..., LeftConstraint('fish', 'science fiction', distance=0), ...,  RightConstraint(x,y), RightConstraint(x,y, distance=0)] |

In [ ]:
import pandas as pd
import numpy as np
import re

columns = ["id", "size", "variables", "Domains", "Constraints"]
parsed = pd.DataFrame(columns=columns)

# df = pd.read_parquet("data/Gridmode-00000-of-00001.parquet")
df = pd.read_parquet("hf://datasets/allenai/ZebraLogicBench/grid_mode/test-00000-of-00001.parquet")


In [ ]:

def parse_var():
  for index, text in enumerate(df["solution"]):
    parsed.loc[index, "variables"] = text.get("header", None)

parse_var()
print(parsed)

      id size                                          variables Domains  \
0    NaN  NaN  [House, Name, Nationality, BookGenre, Food, Co...     NaN   
1    NaN  NaN   [House, Name, Occupation, BookGenre, PhoneModel]     NaN   
2    NaN  NaN        [House, Name, Children, MusicGenre, Height]     NaN   
3    NaN  NaN  [House, Name, Mother, Children, Vacation, Book...     NaN   
4    NaN  NaN                                 [House, Name, Pet]     NaN   
..   ...  ...                                                ...     ...   
995  NaN  NaN               [House, Name, Animal, Cigar, Mother]     NaN   
996  NaN  NaN  [House, Name, Birthday, Mother, Occupation, Ha...     NaN   
997  NaN  NaN                            [House, Name, Vacation]     NaN   
998  NaN  NaN       [House, Name, PhoneModel, Occupation, Color]     NaN   
999  NaN  NaN                        [House, Name, Color, Cigar]     NaN   

    Constraints  
0           NaN  
1           NaN  
2           NaN  
3           NaN

In [60]:
text = df["puzzle"]

for index, problem in enumerate(text):
  vars = parsed.loc[index, "variables"]

  results = []

  i = 0
  if(vars[i] == "House"):
    houses = []
    for h_Count in range(int(df["size"][index][0])):
      houses.append(h_Count)
    i = i+1
    h = {"House": houses}
    results.append(h)

  for line in problem.splitlines():
    if line.strip().startswith("-"):

      names = re.findall(r"`([^`]*)`", line)
      result = {vars[i]: names}

      i = i+1

      results.append(result)

  parsed.loc[index, "Domains"] = results
print(parsed["Domains"])


0      [{'House': [0, 1, 2, 3, 4]}, {'Name': ['Peter'...
1      [{'House': [0, 1, 2, 3]}, {'Name': ['Alice', '...
2      [{'House': [0, 1, 2, 3, 4, 5]}, {'Name': ['Bob...
3      [{'House': [0, 1, 2, 3, 4, 5]}, {'Name': ['Bob...
4      [{'House': [0, 1]}, {'Name': ['Eric', 'Arnold'...
                             ...                        
995    [{'House': [0, 1, 2]}, {'Name': ['Arnold', 'Pe...
996    [{'House': [0, 1, 2, 3, 4]}, {'Name': ['Alice'...
997    [{'House': [0, 1]}, {'Name': ['Arnold', 'Eric'...
998    [{'House': [0, 1]}, {'Name': ['Eric', 'Arnold'...
999    [{'House': [0, 1, 2, 3, 4]}, {'Name': ['Eric',...
Name: Domains, Length: 1000, dtype: object
